# Importing the libraries

In [1]:
import os
import time
import sqlite3
from openai import OpenAI

# Creating and connecting to the Database

In [4]:
conn = sqlite3.connect('Farm_database.db')
c = conn.cursor()
c.execute('''

CREATE TABLE IF NOT EXISTS 
FARM_FACT (PLANT_ID INTEGER PRIMARY KEY, 
           PLANT_NAME TEXT, 
           NS_COORDINATE TEXT, 
           EW_COORDINATE TEXT, 
           SOWING_DT DATE)
''')


# Creating Dummy Data For experimentation purpose

In [7]:
for i in range(150,200):
    data = (str(i), 'Potato', i, i, '2024-03-29')
    c.execute("INSERT INTO FARM_FACT (PLANT_ID, PLANT_NAME, NS_COORDINATE,EW_COORDINATE,SOWING_DT) VALUES (?, ?, ?, ?, ?)", data)
    conn.commit()

# Creating Data Retrival Assistant

In [10]:
client = OpenAI(api_key= '')


assistant = client.beta.assistants.create(
  name="Data Engineer",
    
  instructions = '''
  You are an AI Assistant Data Engineer within the Python notebook. 
  Your Job is to build a custom query every time I ask you a question about my database and nothing else,
  currently, the database has only one table called FARM_FACT and this is its structure:

  FARM_FACT (PLANT_ID INTEGER PRIMARY KEY, 
             PLANT_NAME TEXT, 
             NS_COORDINATE TEXT, 
             EW_COORDINATE TEXT, 
             SOWING_DT DATE)

  PLANT_ID is self-explanatory, and PLANT_NAME is the name of the plant like a tomato plant, grape plant, etc.
  NS_COORDINATE is North-South COORDINATE, EW_COORDINATE is East-West COORDINATE, and SOWING_DT is the date when the seed is sown.

 Remember generate only the query, Nothing else
  
  ''',
    
  tools=[{"type": "code_interpreter"}],
  model="gpt-3.5-turbo",
)

assistant_id = assistant.id

thread = client.beta.threads.create()

thread_id = thread.id

print('assistant_id : ', assistant_id)
print('thread_id : ', thread_id)




assistant_id :  asst_lhX9xIxgEgM3xUsdgsrqEjvA
thread_id :  thread_6O9ICGz5btgM6f72n4cHEK7h


# Asking your Database a question

In [13]:
question = 'how many plants exist in my fields'

In [15]:
message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content=question
)

run = client.beta.threads.runs.create(
  thread_id=thread_id,
  assistant_id=assistant_id,
  
)

while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1) # Wait for 1 second
  run = client.beta.threads.runs.retrieve(
    thread_id=thread_id,
    run_id=run.id
  )
    
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  query = str(messages.data[0].content[0].text.value)
print('query generated: ',query)

query generated:  SELECT COUNT(PLANT_ID) AS TOTAL_PLANTS FROM FARM_FACT


In [17]:
query = 'SELECT COUNT(PLANT_ID) AS TOTAL_PLANTS FROM FARM_FACT'


In [19]:
c.execute(query)
ans = c.fetchall()
print('Data from query', ans)

Data from query [(50,)]
